Import Libraries
TO DO
1) Add larger test dataset
2) Add Speed Tests
3) Create Real Polygons for Testing Purpose

In [32]:
import numpy as np
from osgeo import gdal
from scipy import stats
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
from math import ceil,floor
import matplotlib.pyplot as plt

<img src="data/thumb.png" alt="Drawing" style="width: 300px;"/>

In [33]:
Grid_Name = "data\\nlcd_2011_impervious_2011_edition_2014_10_10.img"
Shapefile_Name = "test\\testPolys.shp"

#r = gdal.Open(Grid_Name)
#g = np.array(r.GetRasterBand(1).ReadAsArray())


In [34]:
class dtFZS:
    def __init__(self, img):

        self.r = gdal.Open(Grid_Name)
        self.g = np.array(self.r.GetRasterBand(1).ReadAsArray())
        
        self.clSze = self.r.GetGeoTransform()[1]
        self.ulX = self.r.GetGeoTransform()[0]
        self.ulY = self.r.GetGeoTransform()[3]
        
        self.brX = (self.g.shape[0] * self.clSze) + self.ulX
        self.brY = self.ulY - (self.g.shape[0] * self.clSze)   
        
        self.geo = self.r.GetGeoTransform()
        
        def dtFZS(g,out):
            for xw in range(0,self.g.shape[1]):
                for yw in range(0,self.g.shape[0]):
                    out[yw,xw] = out[yw-1,xw] + self.g[yw,xw]

                    
        self.data = np.zeros(self.g.shape,dtype='uint64')
        
        dtFZS(self.g,self.data)
        


In [45]:
## Create Class for this grid
dFZS = dtFZS(Grid_Name)

MemoryError: 

Create Helper Functions to move between world and grid cordinates

In [36]:
## Input (world cords; x1, x2, Grid Upper Left, and Cellsize)
## Returns (grid cords; list(x))
## x1 and x2 represent vertexes on a polygon in order

def x_world_to_grid(wx1,wx2,dtFZS):
    
    x1a = min(wx1,wx2)
    x2a = max(wx1,wx2)
    
    xstart = ceil  ((x1a - dtFZS.ulX - (dtFZS.clSze/2)) / dtFZS.clSze)
    xstop =  floor ((x2a - dtFZS.ulX - (dtFZS.clSze/2)) / dtFZS.clSze)

    return(list(range(xstart,xstop+1)))

In [37]:
## Input (grid cords; list(x))
## Returns(world cords centroid ;(list(X)))

def x_grid_to_world(gx,dtFZS):
    return [i * dtFZS.clSze + dtFZS.ulX + (dtFZS.clSze/2) for i in gx]

In [38]:
## Input(world cords centroid ;(list(X)))
## Returns(world cords centroid ;(list(y)))

def y_world_pred(wx1,wx2,wy1,wy2,wxpred):
    slope = 0;
    if (wx1 > wx2):
        wx1,wx2,wy1,wy2 = wx2,wx1,wy2,wy1
    if(wx1 != wx2):    
        slope = (wy2-wy1)/(wx2-wx1)

    return [(slope * (i-wx1)) + wy1 for i in wxpred]

In [39]:
## Input(world cords centroid ;(list(y)))
## Returns (grid cords; list(y))

def y_world_to_grid(wypred,dtFZS):
        return [floor((dtFZS.ulY - i - (dtFZS.clSze/2)) / dtFZS.clSze ) for i in wypred] 

In [40]:
## Input(world Cords for 2 vertexes on a polygon, TL corner of grid, and cellsize)
## Returns(y,x grid cords)

def xy_grid(wx1,wx2,wy1,wy2,dtFZS):
    xgrid  = x_world_to_grid(wx1,wx2,dtFZS)
    wxpred = x_grid_to_world(xgrid,dtFZS)  
    wypred = y_world_pred(wx1,wx2,wy1,wy2,wxpred)
    ygrid  = y_world_to_grid(wypred,dtFZS)
    
    ## Remove negative arrays above
    idx = np.array([True if i >= 0 else False for i in ygrid],dtype=bool)
    
    xgrid = np.array(xgrid)[idx]
    ygrid = np.array(ygrid)[idx]
    
    cords = ((ygrid,xgrid))
    return cords

In [41]:
## Input(list of cords in world, summed down grid, real-world grid tl, cllSze)
## Output Sum v in polygon

def FstZS(geom,DS_FstZS):
    v = geom.exterior.coords  #have not tested
    total = 0;    
    for p in range(1,(len(v)+0)):
        if v[p][0] > v[p-1][0]:
            cords = xy_grid(v[p][0],v[p-1][0],v[p][1],v[p-1][1],DS_FstZS)
            total += np.sum(DS_FstZS.data[cords])
        elif v[p][0] < v[p-1][0]: 
            cords = xy_grid(v[p][0],v[p-1][0],v[p][1],v[p-1][1],DS_FstZS)
            total -= np.sum(DS_FstZS.data[cords])
    return int(total)

In [42]:
def test(geom):
    
    FstZSout = FstZS(geom, dFZS)
    Rastsout = zonal_stats(geom, Grid_Name,stats= "sum")
    
    return (float(FstZSout) , float(Rastsout[0]['sum']) )

In [43]:
## Input(albers conical equal area shapefile input of polygon test features)
## Output polygon geometry for each feature in input shapefile

driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(Shapefile_Name, 0)
layer = dataSource.GetLayer()

for feature in layer:
    geom = feature.GetGeometryRef()
    test(geom)

NameError: name 'dFZS' is not defined

In [44]:
#print(tst)
plt.scatter(y=[t[1] for t in tst],x= [t[0] for t in tst])
plt.xlabel("Fast Zonal Method")
plt.ylabel("Raster Stats")              
plt.show()   

NameError: name 'tst' is not defined